# 🤖 Generate Ukrainian QA Dataset with GPT-4
This notebook generates questions and answers from academic chunks using OpenAI GPT-4.

In [ ]:
!pip install openai tqdm

In [ ]:
import openai
import json
from tqdm import tqdm

# Insert your OpenAI API key
openai.api_key = "sk-..."

In [ ]:
# Load text chunks
with open('/content/chunks.txt', 'r', encoding='utf-8') as f:
    chunks = [line.strip() for line in f if len(line.strip()) > 30]
print(f"Loaded {len(chunks)} chunks.")

In [ ]:
def build_prompt(text):
    return f"""
Ти — освічений мовний асистент. Прочитай наступний фрагмент академічного тексту та згенеруй 2–3 питання з відповідями. Формат відповіді — список об'єктів JSON, кожен з яких має поля "question" та "answer". Текст:

{text}

Поверни лише список JSON-об'єктів.
"""

In [ ]:
def generate_qa(text):
    prompt = build_prompt(text)
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5
        )
        content = response['choices'][0]['message']['content']
        return json.loads(content)
    except Exception as e:
        print(f"Error: {e}")
        return []

In [ ]:
alpaca_data = []
for chunk in tqdm(chunks[:20]):
    qa_pairs = generate_qa(chunk)
    for pair in qa_pairs:
        alpaca_data.append({
            "instruction": "Answer the question based on the text.",
            "input": f"Text: {chunk}\nQuestion: {pair['question']}",
            "output": pair['answer']
        })

with open('qa_dataset.jsonl', 'w', encoding='utf-8') as f:
    for item in alpaca_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print("✅ Saved as qa_dataset.jsonl")